Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [16]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [17]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [9]:
batch_size = 128
hidden_nodes = 1024
reg_weight = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    out1 = tf.matmul(tf_train_dataset, w1) + b1
    
    relu1 = tf.nn.relu(out1)
    
    w2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(relu1, w2) + b2

    # Training computation.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + reg_weight * tf.nn.l2_loss(tf.nn.softmax(logits)))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [19]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 342.784668
Minibatch accuracy: 7.0%
Validation accuracy: 25.5%
Minibatch loss at step 500: 24.790035
Minibatch accuracy: 82.0%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 5.711559
Minibatch accuracy: 78.9%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 7.290588
Minibatch accuracy: 82.8%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 3.859818
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 10.298964
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 2.753796
Minibatch accuracy: 83.6%
Validation accuracy: 82.3%
Test accuracy: 89.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [21]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step % 3) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 328.204987
Minibatch accuracy: 11.7%
Validation accuracy: 28.6%
Minibatch loss at step 500: 0.064000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 1000: 0.064000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 1500: 0.064000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 2000: 0.064000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 2500: 0.064000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 3000: 0.064000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Test accuracy: 81.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [23]:
batch_size = 128
hidden_nodes = 1024
reg_weight = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_prob = tf.placeholder(tf.float32)

    # Variables.
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    out1 = tf.matmul(tf_train_dataset, w1) + b1
    
    relu1 = tf.nn.relu(out1)
    dropout1 = tf.nn.dropout(relu1, keep_prob)
    
    w2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(dropout1, w2) + b2

    # Training computation.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + reg_weight * tf.nn.l2_loss(tf.nn.softmax(logits)))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [31]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step % 3) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1}
            l, predictions = session.run([loss, train_prediction], feed_dict=feed_dict)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 718.351257
Minibatch accuracy: 40.6%
Validation accuracy: 29.0%
Minibatch loss at step 500: 0.198275
Minibatch accuracy: 98.4%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 0.064000
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 0.064000
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.064000
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 2500: 0.064000
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 3000: 0.064000
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Test accuracy: 83.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [42]:
batch_size = 128
hidden_nodes = 1024
reg_weight = 0.001

graph = tf.Graph()

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_prob = tf.placeholder(tf.float32)
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.97)

    # Variables.
    w1 = weight_variable([image_size * image_size, hidden_nodes])
    b1 = bias_variable([hidden_nodes])
    out1 = tf.matmul(tf_train_dataset, w1) + b1
    
    relu1 = tf.nn.relu(out1)
    dropout1 = tf.nn.dropout(relu1, keep_prob)
    
    w2 = weight_variable([hidden_nodes, num_labels])
    b2 = bias_variable([num_labels])
    logits = tf.matmul(dropout1, w2) + b2

    # Training computation.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + reg_weight * tf.nn.l2_loss(tf.nn.softmax(logits)))

    # Optimizer.
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [43]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1}
            l, predictions = session.run([loss, train_prediction], feed_dict=feed_dict)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.900221
Minibatch accuracy: 10.2%
Validation accuracy: 10.6%
Minibatch loss at step 500: 0.595702
Minibatch accuracy: 86.7%
Validation accuracy: 82.2%
Minibatch loss at step 1000: 0.584886
Minibatch accuracy: 87.5%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 0.609007
Minibatch accuracy: 86.7%
Validation accuracy: 84.5%
Minibatch loss at step 2000: 0.522644
Minibatch accuracy: 89.8%
Validation accuracy: 85.2%
Minibatch loss at step 2500: 0.613574
Minibatch accuracy: 86.7%
Validation accuracy: 85.5%
Minibatch loss at step 3000: 0.636985
Minibatch accuracy: 83.6%
Validation accuracy: 85.9%
Minibatch loss at step 3500: 0.596848
Minibatch accuracy: 85.2%
Validation accuracy: 86.2%
Minibatch loss at step 4000: 0.423121
Minibatch accuracy: 89.8%
Validation accuracy: 86.4%
Minibatch loss at step 4500: 0.601208
Minibatch accuracy: 84.4%
Validation accuracy: 86.7%
Minibatch loss at step 5000: 0.425558
Minibatch accuracy: 89.8%
Validation accurac